Create a map with the countries color according to the number of received abstracts.

# Import 

In [1]:
import folium
import json
import pandas as pd
from geopy.geocoders import Nominatim
from geoip import geolite2
import pycountry
import re
from collections import Counter 

/usr/local/lib/python2.7/dist-packages/matplotlib/__init__.py:913: UserWarning: axes.color_cycle is deprecated and replaced with axes.prop_cycle; please use the latter.
  warnings.warn(self.msg_depr % (key, alt_key))


* [Folium](https://pypi.python.org/pypi/folium) to visualise with Leaflet,
* [json](https://docs.python.org/2/library/json.html) to read country information,
* [pandas](http://pandas.pydata.org/) to read the abstract data,
* [geopy](https://pypi.python.org/pypi/geopy) to get coordinates from location,
* [geopip](https://pypi.python.org/pypi/GeoIP/) to get the location from the IP address,
* [pycountry](https://pypi.python.org/pypi/pycountry) to change the country name into country code.

In [2]:
print(folium.__version__)

0.2.1


# Files I/O

## Input files

Two input files are necessary: 
* abstractfile, which contains a list of JSON files corresponding to the submitted abstract (can be obtained with ls json > abstractfile);
* geo_json_data, a file containing the country boundaries in JSON format. Can be obtained from https://github.com/johan/world.geo.json.

In [4]:
abstractfile = '../data/abstractslist2016.txt'
geo_json_data = r'../data/countries.geo.json'

## Output file

* mapfile, an html file showing the final map. 

In [6]:
match = re.search(r'\d{4}', abstractfile)
year = match.group()
mapfile = '../html/CountryAbstract{0}.html'.format(year)
print(mapfile)

../html/CountryAbstract2016.html


In [7]:
geolocator = Nominatim()
location = geolocator.geocode("Place du XX Aout, Liege, Belgium")

# Create a list of countries from the list of abstracts

See [plot_abstracts_map.ipynb](./plot_abstracts_map.ipynb) for more details on this part.

In [8]:
countrylist = []
nlines = 0
with open(abstractfile, 'r') as f:
    for lines in f.readlines():
        nlines += 1
        match = re.search(r'(\d{4})-(\d{2})-(\d{2})_(\d{2}):(\d{2}):(\d{2})_IP_(\d+\.\d+\.\d+\.\d+).json', lines)
        if match:
            IP = match.group(7)
            matchIP = geolite2.lookup(IP)
            if matchIP: 
                countrylist.append(matchIP.country)
abstract_countrycount = Counter(countrylist)

Counter({'AU': 4,
         'BE': 25,
         'BR': 2,
         'BZ': 1,
         'CA': 6,
         'CH': 1,
         'CL': 3,
         'CM': 1,
         'CN': 3,
         'DE': 19,
         'DK': 1,
         'EE': 4,
         'ES': 8,
         'FI': 1,
         'FR': 33,
         'GB': 18,
         'GF': 1,
         'GR': 1,
         'HR': 1,
         'IL': 1,
         'IN': 14,
         'IR': 3,
         'IT': 4,
         'JP': 3,
         'KE': 1,
         'KR': 4,
         'LT': 1,
         'MA': 1,
         'MX': 1,
         'NG': 1,
         'NL': 1,
         'NO': 3,
         'PT': 3,
         'RU': 6,
         'SA': 6,
         'TG': 1,
         'TR': 1,
         'UA': 2,
         'US': 59,
         'VN': 2,
         'ZA': 5})

We get an object that stores the number of abstracts submitted per country.

In [9]:
abstract_countrycount

collections.Counter

Change the dictionary into a data file in CSV format (loop on the keys and values of the dictionnary; probably another way to do it).

In [10]:
countryfile = 'countryabstract' + year + '.csv'
with open(countryfile, 'w') as f:
    f.write('Country, Abstracts\n')
    for ii, jj in zip(abstract_countrycount.keys(), abstract_countrycount.values()):
        countrycode = pycountry.countries.get(alpha2=ii).alpha3
        f.write(countrycode + ',' + str(jj) + '\n')

Read the abstract data using [Pandas](http://pandas.pydata.org/):

In [11]:
abstract_data = pd.read_csv(countryfile)

# Create the map

We initialize the map with the center on Belgium and with a global view (zoom = 1.5).

In [12]:
map_clq = folium.Map(location=[location.latitude, location.longitude], 
                     zoom_start=1.5)
map_clq.simple_marker([location.latitude, location.longitude], popup='Colloquium Venue')
#map_clq.geo_json('countries.geo.json')

/usr/local/lib/python2.7/dist-packages/ipykernel/__main__.py:3: FutureWarning: simple_marker is deprecated. Use add_children(Marker) instead
  app.launch_new_instance()


Then we add the information on the abstracts:

In [13]:
map_clq.geo_json(geo_path=geo_json_data, data=abstract_data,
             columns=['Country','Abstracts'],
             key_on='feature.id',
             fill_color='YlGn', fill_opacity=0.5, line_opacity=0.2,
             legend_name='Number of submitted abstracts',
             reset='True')
# map_clq.lat_lng_popover()
map_clq.create_map(mapfile)

/usr/local/lib/python2.7/dist-packages/folium/folium.py:504: UserWarning: This method is deprecated. Please use Map.choropleth instead.
  warnings.warn('This method is deprecated. '


KeyError: 'Abstracts'

In [11]:
map_clq